In [ ]:
# Save as app.py and run with: streamlit run app.py

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Logistic Regression Scoring Function ---
def compute_logistic_score(row):
    score = -1.2091 \
        + (0.0872 * row.get("Genre_Femme", 0)) \
        + (0.1478 * row.get("Genre_Homme", 0)) \
        + (-1.9765 * row.get("Moyen d'obtention de salaire_Directement auprès de l'opérateur", 0)) \
        + (2.2474 * row.get("Moyen d'obtention de salaire_Virement bancaire", 0)) \
        + (0.0418 * row.get("Type du crédit demandé_Crédit de consommation", 0)) \
        + (0.0194 * row.get("Type du crédit demandé_Crédit immobilier", 0)) \
        + (0.0259 * row.get("Type du crédit demandé_Crédit- auto", 0)) \
        + (0.6370 * row.get("Avez vous un crédit en cours?_Crédit pour la première fois", 0)) \
        + (-0.3333 * row.get("Avez vous un crédit en cours?_J'ai déjà un crédit bancaire et je veux un autre", 0)) \
        + (-0.0716 * row.get("Avez vous un crédit en cours?_L'ancien crédit est remboursé et je veux un autre", 0)) \
        + (0.1188 * row.get("Type du crédit obtenu_Crédit au démarrage du projet", 0)) \
        + (0.0709 * row.get("Type du crédit obtenu_Crédit de consommation", 0)) \
        + (-0.0244 * row.get("Type du crédit obtenu_Crédit immobilier", 0)) \
        + (-0.0036 * row.get("Type du crédit obtenu_Crédit- auto", 0)) \
        + (0.0293 * row.get("Type du crédit obtenu_Micro-crédit", 0)) \
        + (0.0064 * row.get("Type du crédit obtenu_crédit de consommation", 0)) \
        + (0.6924 * row.get("Type du crédit obtenu_pas de crédit obtenu", 0)) \
        + (-3.1489 * row.get("Avez-vous dépassé la moitié de la période totale de remboursement_non", 0)) \
        + (2.6378 * row.get("Avez-vous dépassé la moitié de la période totale de remboursement_oui", 0)) \
        + (0.6924 * row.get("Avez-vous dépassé la moitié de la période totale de remboursement_pas de crédit obtenu", 0)) \
        + (0.2040 * row.get("Avez-vous des cessions sur salaire?_non", 0)) \
        + (0.0718 * row.get("Avez-vous des cessions sur salaire?_oui", 0)) \
        + (1.6208 * row.get("Income Category_Good Income", 0))
    return score

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def classify_approval(prob):
    if prob < 0.50:
        return "❌ Not Approved"
    elif prob < 0.75:
        return "⏳ Under Consideration"
    else:
        return "✅ Approved"

st.title("💳 Credit Approval Scoring System")

# --- Single Client Evaluation Form ---
st.header("📋 Evaluate One Client Manually")
with st.form("client_form"):
    gender = st.selectbox("Genre", ["Femme", "Homme"])
    salaire = st.selectbox("Moyen d'obtention de salaire", [
        "Directement auprès de l'opérateur", "Virement bancaire"])
    type_credit_demande = st.selectbox("Type du crédit demandé", [
        "Crédit de consommation", "Crédit immobilier", "Crédit- auto"])
    credit_status = st.selectbox("Avez-vous un crédit en cours ?", [
        "Crédit pour la première fois",
        "J'ai déjà un crédit bancaire et je veux un autre",
        "L'ancien crédit est remboursé et je veux un autre"
    ])
    type_credit_obtenu = st.selectbox("Type du crédit obtenu", [
        "Crédit au démarrage du projet", "Crédit de consommation",
        "Crédit immobilier", "Crédit- auto", "Micro-crédit",
        "crédit de consommation", "pas de crédit obtenu"
    ])
    duree_remboursement = st.selectbox("Avez-vous dépassé la moitié de la période de remboursement ?", [
        "non", "oui", "pas de crédit obtenu"
    ])
    cession_salaire = st.selectbox("Avez-vous des cessions sur salaire ?", ["oui", "non"])
    income_cat = st.selectbox("Catégorie de revenu", ["Good Income", "Other"])

    submitted = st.form_submit_button("🔍 Évaluer")

if submitted:
    row = {
        f"Genre_{gender}": 1,
        f"Moyen d'obtention de salaire_{salaire}": 1,
        f"Type du crédit demandé_{type_credit_demande}": 1,
        f"Avez vous un crédit en cours?_{credit_status}": 1,
        f"Type du crédit obtenu_{type_credit_obtenu}": 1,
        f"Avez-vous dépassé la moitié de la période totale de remboursement_{duree_remboursement}": 1,
        f"Avez-vous des cessions sur salaire?_{cession_salaire}": 1,
        "Income Category_Good Income": 1 if income_cat == "Good Income" else 0
    }

    score = compute_logistic_score(row)
    prob = sigmoid(score)
    decision = classify_approval(prob)

    st.subheader("🔎 Evaluation Results")
    st.metric("Score Logistique", f"{score:.2f}")
    st.metric("Probabilité d'approbation", f"{prob*100:.2f} %")
    st.metric("Décision", decision)

# --- Batch Scoring for Uploaded Dataset ---
st.header("📂 Upload CSV for Batch Scoring (with actual labels optional)")
uploaded = st.file_uploader("Upload encoded test CSV", type=["csv"])
if uploaded:
    df = pd.read_csv(uploaded)

    # Compute scores
    df["score"] = df.apply(compute_logistic_score, axis=1)
    df["approval_probability"] = sigmoid(df["score"])
    df["approval_score_percent"] = df["approval_probability"] * 100
    df["model_decision"] = df["approval_probability"].apply(classify_approval)

    # Compare to actual if available
    if "Final credit result" in df.columns:
        df["actual_result"] = df["Final credit result"].map({1: "✅ Approved", 0: "❌ Not Approved"})
        st.subheader("📊 Model vs. Actual")
        st.dataframe(df[["approval_score_percent", "model_decision", "actual_result"]].head())

        # Confusion matrix
    
        st.markdown("### 📉 Confusion Matrix")
        from sklearn.metrics import confusion_matrix
        import seaborn as sns

        y_true = df["Final credit result"]
        y_pred = (df["approval_probability"] >= 0.75).astype(int)  # model "approved"
        cm = confusion_matrix(y_true, y_pred)

        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Not Approved", "Approved"], yticklabels=["Not Approved", "Approved"])
        ax.set_xlabel("Predicted")
        ax.set_ylabel("Actual")
        st.pyplot(fig)

    # Download
    st.subheader("📥 Download Scored Dataset")
    csv = df.to_csv(index=False).encode("utf-8")
    st.download_button("Download CSV", csv, file_name="scored_clients.csv", mime="text/csv")
